데이터 셋 다운로드

In [1]:
import os
import tarfile
import urllib

down_root = 'https://raw.githubusercontent.com/ageron/handson-ml2/master/'
housing_path = os.path.join('datasets', 'housing')
housing_url = down_root + 'datasets/housing/housing.tgz'

def fetch_housing_data(housing_url = housing_url, housing_path = housing_path) : 
    os.makedirs(housing_path, exist_ok = True)
    tgz_path = os.path.join(housing_path, 'housing.tgz')
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path = os.path.join('data'))
    housing_tgz.close()


In [2]:
fetch_housing_data()

데이터 셋 불러오기

In [3]:
import pandas as pd

def load_data(data_path, data_name) : 
    csv_path = os.path.join(data_path, data_name)
    return pd.read_csv(csv_path)

data_path = os.path.join('data')
data_name = 'housing.csv'

housing = load_data(data_path = data_path, data_name = data_name)

In [4]:
housing['ocean_proximity'].value_counts()

<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: ocean_proximity, dtype: int64

## 파이프라인 구성

In [12]:
from sklearn import pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np


# 열 인덱스
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True): # *args 또는 **kargs 없음
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # 아무것도 하지 않습니다
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [13]:
#숫자만 진행해준다.
housing_num = housing.drop('ocean_proximity', axis = 1)

#파이프라인 구성!
num_pipeline = pipeline.Pipeline([
    ('imputer', SimpleImputer(strategy = 'median')), #누락된 값을 중간값으로 대체한다.
    ('attrib_adder', CombinedAttributesAdder()), #새로운 파생변수를 만든다.
    ('std_scaler', StandardScaler()), #이후 정규화를 진행한다.
])

housing_num_tr = num_pipeline.fit_transform(housing_num)
housing_num_tr_fit = num_pipeline.fit(housing_num)

In [16]:
print(housing_num)

#보아하니 fit_transform하면 fit해주고 변환까지 해주는 과정인 것 같다.

       longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0        -122.23     37.88                41.0        880.0           129.0   
1        -122.22     37.86                21.0       7099.0          1106.0   
2        -122.24     37.85                52.0       1467.0           190.0   
3        -122.25     37.85                52.0       1274.0           235.0   
4        -122.25     37.85                52.0       1627.0           280.0   
...          ...       ...                 ...          ...             ...   
20635    -121.09     39.48                25.0       1665.0           374.0   
20636    -121.21     39.49                18.0        697.0           150.0   
20637    -121.22     39.43                17.0       2254.0           485.0   
20638    -121.32     39.43                18.0       1860.0           409.0   
20639    -121.24     39.37                16.0       2785.0           616.0   

       population  households  median_income  media

In [20]:
print(housing_num_tr_fit)
#보아하니 여기는 마지막 변환기와 추정기에 fit만 먹여준 상태!
#아마 모델같은 경우는 fit한 다음에 predict를 해주는게 크니까, 모델같은 경우는 마지막에 fit해주면 될 것 같고,,, 
#transform은 마지막이 변환기일 경우에 써주면 될 것 같다.

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('attrib_adder', CombinedAttributesAdder()),
                ('std_scaler', StandardScaler())])


### Pipeline 설명!

> 마지막 단계는 추정기/변환기 아무거나 상관없고 중간 단계는 변환기로만 이루어 져야 한다.

> fit()메서드를 호출하면 모든 변환기의 fit_transform()메서드를 순서대로 호출! 마지막 단계는 fit() 메서드만 호출한다.

추정기와 변환기란?

> 추정기 : 모델 파라미터를 추정하는 것은 추정기! 추정 자체는 fit()메서드에 의해서 실행된다.하나의 매개변수로 하나의 데이터 셋만 전달된다.

> 변환기 : 데이터셋을 변환하는 추정기를 변환기라고 한다. transform()메서드가 수행하고, 변환된 데이터 셋을 반환한다. fit_transform()메서드도 있다고 한다. 아마 추정기를 기반으로 하므로 fit 메서드도 있을 것 같다